In [ ]:
import os
import numpy as np
from skimage import io, color
from scipy.stats import kurtosis, skew, entropy
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from PIL import Image
import math

def get_media(imagem):
    largura, altura = imagem.size
    soma_pixel = 0
    pixels = imagem.load()

    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += sum(pixel)
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += pixel  # Imagem em tons de cinza tem apenas um valor de pixel

    media = soma_pixel / (largura * altura)

    return media


def get_variancia(imagem):
    largura, altura = imagem.size
    soma_pixel = 0
    soma_quadrados = 0
    pixels = imagem.load()
    
    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += sum(pixel)
                soma_quadrados += sum(componente ** 2 for componente in pixel)
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += pixel
                soma_quadrados += pixel ** 2

    media = soma_pixel / (largura * altura)
    variancia = (soma_quadrados / (largura * altura)) - (media ** 2)
    
    return variancia


def get_skewness(imagem):
    largura, altura = imagem.size
    soma_pixel = 0
    soma_quadrados = 0
    soma_cubo = 0
    pixels = imagem.load()

    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += sum(pixel)
                soma_quadrados += sum(componente ** 2 for componente in pixel)
                soma_cubo += sum(componente ** 3 for componente in pixel)
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += pixel
                soma_quadrados += pixel ** 2
                soma_cubo += pixel ** 3

    media = soma_pixel / (largura * altura)

    variancia = (soma_quadrados / (largura * altura)) - (media ** 2)

    skewness = (soma_cubo / (largura * altura)) - 3 * media * variancia - media ** 3

    return skewness


def get_kurtosis(imagem):

    largura, altura = imagem.size
    soma_pixel = 0
    soma_quadrados = 0
    soma_quarta_potencia = 0
    pixels = imagem.load()

    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += sum(pixel)
                soma_quadrados += sum(componente ** 2 for componente in pixel)
                soma_quarta_potencia += sum(componente ** 4 for componente in pixel)
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_pixel += pixel
                soma_quadrados += pixel ** 2
                soma_quarta_potencia += pixel ** 4
            
    media = soma_pixel / (largura * altura)

    variancia = (soma_quadrados / (largura * altura)) - (media ** 2)

    kurtosis = (soma_quarta_potencia / (largura * altura)) - 3 * (variancia ** 2)

    return kurtosis


def get_energia(imagem):
    largura, altura = imagem.size
    soma_quadrados = 0
    pixels = imagem.load()

    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                for componente in pixel:
                    soma_quadrados += componente ** 2
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                soma_quadrados += pixel ** 2

    energia = soma_quadrados

    return energia


def get_entropia(imagem):
    largura, altura = imagem.size
    frequencia_pixel = {}

    pixels = imagem.load()

    if imagem.mode == 'RGB':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                frequencia_pixel[pixel] = frequencia_pixel.get(pixel, 0) + 1
    elif imagem.mode == 'L':
        for x in range(largura):
            for y in range(altura):
                pixel = pixels[x, y]
                frequencia_pixel[pixel] = frequencia_pixel.get(pixel, 0) + 1

    entropia = 0.0
    total_pixels = largura * altura

    for valor_pixel, frequencia in frequencia_pixel.items():
        probabilidade = frequencia / total_pixels
        entropia -= probabilidade * math.log2(probabilidade)

    return entropia


def descritor_histograma(imagem):
    
    media = get_media(imagem)
    var = get_variancia(imagem)
    sk = get_skewness(imagem)
    kur = get_kurtosis(imagem)
    ener = get_energia(imagem)
    entr = get_entropia(imagem)

    return [media,var,sk,kur,ener,entr]


In [ ]:
# Diretório onde as imagens estão armazenadas
dataset_path = os.path.join(os.getcwd(), 'datasets', 'animals')

# Categorias de animais
categorias = ['dogs', 'panda', 'cats']

# Número de imagens a serem exibidas por categoria
num_images_to_display = 9

# Loop para percorrer as categorias e exibir algumas imagens de cada categoria
for categoria in categorias:
    fig, axes = plt.subplots(1, num_images_to_display, figsize=(12, 4))
    fig.suptitle(f'Imagens da categoria {categoria}')
    
    # Diretório da categoria
    diretorio_categoria = os.path.join(dataset_path, categoria)
    imagens_categoria = os.listdir(diretorio_categoria)
    
    for i in range(num_images_to_display):
        # Carregar a imagem
        imagem = io.imread(os.path.join(diretorio_categoria, imagens_categoria[i]))
        axes[i].imshow(imagem)
        axes[i].set_title(f'Imagem {i+1}')
        axes[i].axis('off')
    
    plt.show()

In [ ]:
# Lista para armazenar os atributos e rótulos
atributos = []
rotulos = []

# Loop para percorrer as categorias
for categoria in categorias:
    diretorio_categoria = os.path.join(dataset_path, categoria)
    for nome_arquivo in os.listdir(diretorio_categoria):
        # Caminho completo da imagem
        caminho_imagem = os.path.join(diretorio_categoria, nome_arquivo)
        # Carregar a imagem
        imagem = Image.open(caminho_imagem)
        
        print(imagem)
        
        # Calcular os atributos de textura
        atributos_textura = descritor_histograma(imagem)
        # Adicionar os atributos à lista
        atributos.append(atributos_textura)
        # Adicionar o rótulo de classe (0 para dogs, 1 para panda, 2 para cats)
        rotulos.append(categorias.index(categoria))

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(atributos, rotulos, test_size=0.3, random_state=42)

# Criar um classificador MLP (rede neural)
clf = MLPClassifier()
clf.fit(X_train, y_train)  # Treinar o classificador

# Realizar a predição no conjunto de teste
y_pred = clf.predict(X_test)

# Calcular a acurácia
acuracia = accuracy_score(y_test, y_pred)

# Exibir a acurácia
print(f'Acurácia da classificação: {acuracia:.2f}')

In [ ]:
# Scatter plot dos atributos (apenas um exemplo)
plt.scatter([x[0] for x in atributos], [x[1] for x in atributos], c=rotulos, cmap='viridis')
plt.xlabel('Média')
plt.ylabel('Variância')
plt.show()

In [ ]:
# Scatter plots dos atributos (combinando todos os pares)
atributos = np.array(atributos)
atributos_nomes = ["Média", "Variância", "Skewness", "Kurtosis", "Energia", "Entropia"]

# Plotar scatter plots de todos os pares de atributos
plt.figure(figsize=(15, 15))
for i in range(6):
    for j in range(i+1, 6):
        plt.subplot(6, 5, i*5 + j + 1)
        plt.scatter(atributos[:, i], atributos[:, j], c=rotulos, cmap='viridis')
        plt.xlabel(atributos_nomes[i])
        plt.ylabel(atributos_nomes[j])
plt.show()